In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import csv
import numpy as np
from sklearn.metrics import f1_score

### Data Setup

In [9]:
all_continents = ['afr', 'am', 'asia', 'eur', 'me']
all_data = {}

feature_names = ["warstds", "ager", "agexp", "anoc", "army85", "autch98", "auto4",
"autonomy", "avgnabo", "centpol3", "coldwar", "decade1", "decade2",
"decade3", "decade4", "dem", "dem4", "demch98", "dlang", "drel",
"durable", "ef", "ef2", "ehet", "elfo", "elfo2", "etdo4590",
"expgdp", "exrec", "fedpol3", "fuelexp", "gdpgrowth", "geo1", "geo2",
"geo34", "geo57", "geo69", "geo8", "illiteracy", "incumb", "infant",
"inst", "inst3", "life", "lmtnest", "ln_gdpen", "lpopns", "major", "manuexp", "milper",
"mirps0", "mirps1", "mirps2", "mirps3", "nat_war", "ncontig",
"nmgdp", "nmdp4_alt", "numlang", "nwstate", "oil", "p4mchg",
"parcomp", "parreg", "part", "partfree", "plural", "plurrel",
"pol4", "pol4m", "pol4sq", "polch98", "polcomp", "popdense",
"presi", "pri", "proxregc", "ptime", "reg", "regd4_alt", "relfrac", "seceduc",
"second", "semipol3", "sip2", "sxpnew", "sxpsq", "tnatwar", "trade",
"warhist", "xconst"]

for cont in all_continents:
    for ver in ['tr', 'test']:
        data = []
        names = []
        with open('{0}_{1}.csv'.format(cont, ver), newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for i, row in enumerate(reader):
                if i == 0:
                    names = row
                else:
                    data.append(row)
        data = np.array(data)
        feature_indices = [names.index(feat) for feat in feature_names[1:]]
        features = data[:, feature_indices]
        features = features.astype(float)
        labels = data[:, names.index('warstds')]
        labels = [int(float(label)) for label in labels]
        all_data['{0}_{1}'.format(cont, ver)] = [features, labels]
        

In [88]:
# hyperparameters
rdm_depth = 5
reg = 1000

### Baseline: Train on all continents, disaggregated test results on each continent

In [89]:
# train on all data
all_train_features = []
all_train_labels = []
for key in all_data.keys():
    if 'tr' in key:
        all_train_features.append(all_data[key][0])
        all_train_labels.append(all_data[key][1])
all_train_features = np.concatenate(all_train_features, axis=0)
all_train_labels = np.concatenate(all_train_labels, axis=0)

log_reg = LogisticRegression(C=reg, class_weight='balanced').fit(features, labels)
log_reg_score = log_reg.score(features, labels)
rdm_for = RandomForestClassifier(max_depth=rdm_depth, class_weight='balanced').fit(features, labels)
rdm_for_score = rdm_for.score(features, labels)
print("Training Scores for Logistic Regression: {0}, Random Forest: {1}\n".format(log_reg_score, rdm_for_score))
baseline_accs = []

for key in all_data.keys():
    if 'test' in key:
        test_features = all_data[key][0]
        test_labels = all_data[key][1]
        log_reg_score = log_reg.score(test_features, test_labels)
        rdm_for_score = rdm_for.score(test_features, test_labels)
        log_reg_f1score = f1_score(test_labels, log_reg.predict(test_features))
        rdm_for_f1score = f1_score(test_labels, rdm_for.predict(test_features))
        baseline_accs.append([log_reg_f1score, rdm_for_f1score])
        print("Test on {0}, LR: {1}, RF: {2}".format(key.split('_')[0], round(log_reg_score, 5), round(rdm_for_score, 5)))
        print("F1 for LR: {0}, RF: {1}\n".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
        

Training Scores for Logistic Regression: 0.9653739612188366, Random Forest: 0.9944598337950139

Test on eur, LR: 0.97811, RF: 0.99375
F1 for LR: 0.0, RF: 0.1667

Test on me, LR: 0.96537, RF: 0.99446
F1 for LR: 0.5614, RF: 0.8824

Test on afr, LR: 0.66684, RF: 0.95994
F1 for LR: 0.0574, RF: 0.092

Test on am, LR: 0.48683, RF: 0.94733
F1 for LR: 0.0281, RF: 0.1136

Test on asia, LR: 0.82284, RF: 0.95095
F1 for LR: 0.0692, RF: 0.2299



### Exp: Train on all continents except for x, and then test on x

In [90]:
oos_accs = []
for key in all_data.keys():
    if 'test' in key:
        cont = key.split('_')[0]
        
        train_continents = all_continents.copy()
        train_continents.remove(cont)
        all_train_features = []
        all_train_labels = []
        for t_cont in train_continents:
            all_train_features.append(all_data['{0}_tr'.format(t_cont)][0])
            all_train_labels.append(all_data['{0}_tr'.format(t_cont)][1])
        all_train_features = np.concatenate(all_train_features, axis=0)
        all_train_labels = np.concatenate(all_train_labels, axis=0)

        log_reg = LogisticRegression(C=reg, class_weight='balanced').fit(features, labels)
        rdm_for = RandomForestClassifier(max_depth=rdm_depth, class_weight='balanced').fit(features, labels)
        
        test_features = all_data[key][0]
        test_labels = all_data[key][1]
        log_reg_score = log_reg.score(test_features, test_labels)
        rdm_for_score = rdm_for.score(test_features, test_labels)
        log_reg_f1score = f1_score(test_labels, log_reg.predict(test_features))
        rdm_for_f1score = f1_score(test_labels, rdm_for.predict(test_features))
        oos_accs.append([log_reg_f1score, rdm_for_f1score])
        
        print("Extrap to {0}, LR: {1}, RF: {2}".format(cont, round(log_reg_score, 5), round(rdm_for_score, 5)))
        print("F1 for LR: {0}, RF: {1}\n".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))

Extrap to eur, LR: 0.97811, RF: 0.99124
F1 for LR: 0.0, RF: 0.0

Extrap to me, LR: 0.96953, RF: 0.99307
F1 for LR: 0.5926, RF: 0.8485

Extrap to afr, LR: 0.63489, RF: 0.9645
F1 for LR: 0.0551, RF: 0.0541

Extrap to am, LR: 0.52937, RF: 0.98717
F1 for LR: 0.0279, RF: 0.0952

Extrap to asia, LR: 0.84407, RF: 0.9407
F1 for LR: 0.0533, RF: 0.2136



### Compare Accuracy Drops

In [91]:
i = 0
for key in all_data.keys():
    if 'test' in key:
        cont = key.split('_')[0]
        print("Drop in {0}: LR: {1}, RF: {2}".format(cont, round(baseline_accs[i][0] - oos_accs[i][0], 4), round(baseline_accs[i][1] - oos_accs[i][1], 4)))
        i += 1

Drop in eur: LR: 0.0, RF: 0.1667
Drop in me: LR: -0.0312, RF: 0.0339
Drop in afr: LR: 0.0023, RF: 0.0379
Drop in am: LR: 0.0002, RF: 0.0184
Drop in asia: LR: 0.0159, RF: 0.0163
